# Como um Agent é construído

A ideia central dos agentes é usar um modelo de linguagem para escolher uma sequência de ações a serem executadas. Em cadeias, uma sequência de ações é codificada diretamente (no código). Em agentes, um modelo de linguagem é usado como um motor de raciocínio para determinar quais ações devem ser tomadas e em qual ordem.

## Criando as tools que usaremos

## Revisando a utilização das tools

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 131, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6964edf8-a350-45c3-a85d-348117e6809b-0', usage_metadata={'input_tokens': 131, 'output_tokens': 28, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache

'retorna_temperatura_atual'

{'latitude': -27.5954, 'longitude': -48.548}

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 131, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6964edf8-a350-45c3-a85d-348117e6809b-0', usage_metadata={'input_tokens': 131, 'output_tokens': 28, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

## Adicionando o racicínio do agent as mensagens (agent_scretchpad)

Temos que adicionar junto as nossas mensagens um campo que armazenará o raciocínio atual do modelo chamado agent_scratchpad. Para isso, utilizamos um MessagesPlaceholder ao nosso prompt. Ele guardará espaço para o raciocínio e, caso o modelo não esteja gerando um raciocínio no momento, o MessagesPlaceholder não é utilizado.

AgentActionMessageLog(tool='retorna_temperatura_atual', tool_input={'latitude': -27.5954, 'longitude': -48.548}, log="\nInvoking: `retorna_temperatura_atual` with `{'latitude': -27.5954, 'longitude': -48.548}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 131, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0ebea0e8-befb-435d-9f9d-e5025622d294-0', usage_metadata={'input_tokens': 131, 'output_tokens': 28, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache

'30.2ºC'

Podemos utilizar a função format_to_openai_function_messages para modificar o formato da resposta de forma que ela possa ser enviada, junto da observação, de volta ao modelo. No caso o que está ocorrendo é o modelo está pedindo que uma tool seja rodada, estamos rodando a tool e gerando uma observação, e a gora enviamos novamente para o modelo a pergunta original, a mensagem do próprio modelo dizendo que precisava que a tool fosse rodada e a observação gerada pela ferramenta.

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-27.5954,"longitude":-48.548}', 'name': 'retorna_temperatura_atual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 131, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0ebea0e8-befb-435d-9f9d-e5025622d294-0', usage_metadata={'input_tokens': 131, 'output_tokens': 28, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='30.2ºC', additional_kwargs={}, response_metadata={}, name='retorna_temperatura_atual')]

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 30.2ºC.'}, log='A temperatura atual em Florianópolis é de 30.2ºC.')

### Criando um loop de racicínio

Por fim, podemos criar um loop que adiciona a automaticamente as chamadas de função e observações e fica chamando o modelo novamente até que a mensagem de AgentFinish seja recebida.

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 30.2ºC.'}, log='A temperatura atual em Florianópolis é de 30.2ºC.')

Modificamos um pouco o formato para padronizar ao funcionamento do LangChain para agents

{'input': 'Qual é a temperatura de Floripa?',
 'intermediate_steps': [],
 'agent_scratchpad': []}

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 30.2ºC.'}, log='A temperatura atual em Florianópolis é de 30.2ºC.')

## O que temos no final?

### Um Agent

### Um AgentExecutor